# 1. Train and predict with first model
model_1.fit(X_train_1, y_train)
y_pred_1 = model_1.predict(X_train)

# 2. Train and predict with second model on residuals
model_2.fit(X_train_2, y_train - y_pred_1)
y_pred_2 = model_2.predict(X_train_2)

# 3. Add to get overall predictions
y_pred = y_pred_1 + y_pred_2

In [7]:
class BoostedHybrid:
    def __init__(self, model_1, model_2):
        self.model_1 = model_1
        self.model_2 = model_2
        self.y_columns = None

        BoostedHybrid.fit = fit
        BoostedHybrid.predict = predict

    def fit(self, X_1, X_2, y):
        # Train model_1
        self.model_1.fit(X_1, y)
    
        # Make predictions
        y_fit = pd.DataFrame(
            self.model_1.predict(X_1), 
            index=X_1.index, columns=y.columns)
    
        # Compute residuals
        y_resid = y - y_fit
        y_resid = y_resid.stack().squeeze() # wide to long
    
        # Train model_2 on residuals
        self.model_2.fit(X_2, y_resid)
    
        # Save column names for predict method
        self.y_columns = y.columns
        # Save data for question checking
        self.y_fit = y_fit
        self.y_resid = y_resid

    def predict(self, X_1, X_2):
        # Predict with model_1
        y_pred = pd.DataFrame(
            self.model_1.predict(X_1), 
            index=X_1.index, columns=self.y_columns)
        y_pred = y_pred.stack().squeeze()  # wide to long
    
        # Add model_2 predictions to model_1 predictions
        y_pred += self.model_2.predict(X_2)
    
        return y_pred.unstack()


In [14]:
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.linear_model import LinearRegression
from statsmodels.tsa.deterministic import DeterministicProcess
# from xgboost import XGBRegressor